In [78]:
import datetime, math
from pyspark.context import SparkContext
'''Random Forest'''
from pyspark.ml.regression import RandomForestRegressor
from pyspark.mllib.linalg import Vectors
from pyspark.sql import DataFrame

from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.util import MLUtils

In [79]:
sc = SparkContext.getOrCreate()

In [80]:
data_path = "./properties_price_12k.csv"
data_set = spark.read.options(header="true", parserLib="univocity", inferSchema="true").csv(data_path)
print data_set.schema
headers = data_set.columns
print(headers)
print("\n")
print 'headers names and types: \n'
print data_set.dtypes

StructType(List(StructField(_c0,IntegerType,true),StructField(Parcel ID,StringType,true),StructField(Lat,DoubleType,true),StructField(Lng,DoubleType,true),StructField(Property class,IntegerType,true),StructField(Year built,IntegerType,true),StructField(Square feet,IntegerType,true),StructField(Bedrooms,IntegerType,true),StructField(Bathrooms,DoubleType,true),StructField(ZIP code,IntegerType,true),StructField(Transaction year,IntegerType,true),StructField(Transaction price,IntegerType,true)))
['_c0', 'Parcel ID', 'Lat', 'Lng', 'Property class', 'Year built', 'Square feet', 'Bedrooms', 'Bathrooms', 'ZIP code', 'Transaction year', 'Transaction price']


headers names and types: 

[('_c0', 'int'), ('Parcel ID', 'string'), ('Lat', 'double'), ('Lng', 'double'), ('Property class', 'int'), ('Year built', 'int'), ('Square feet', 'int'), ('Bedrooms', 'int'), ('Bathrooms', 'double'), ('ZIP code', 'int'), ('Transaction year', 'int'), ('Transaction price', 'int')]


In [81]:
data_set.take(5)

[Row(_c0=0, Parcel ID=u'429-01-002', Lat=37.3084232, Lng=-121.8951796, Property class=0, Year built=1900, Square feet=1327, Bedrooms=4, Bathrooms=2.0, ZIP code=95125, Transaction year=2015, Transaction price=917184),
 Row(_c0=1, Parcel ID=u'429-01-003', Lat=37.3085298, Lng=-121.895274, Property class=0, Year built=1947, Square feet=1575, Bedrooms=3, Bathrooms=2.0, ZIP code=95125, Transaction year=2011, Transaction price=442886),
 Row(_c0=2, Parcel ID=u'429-01-005', Lat=37.3088046, Lng=-121.8955309, Property class=0, Year built=1905, Square feet=1360, Bedrooms=3, Bathrooms=1.0, ZIP code=95125, Transaction year=2015, Transaction price=1250432),
 Row(_c0=3, Parcel ID=u'429-01-007', Lat=37.3089809, Lng=-121.8956939, Property class=0, Year built=1905, Square feet=1044, Bedrooms=3, Bathrooms=1.0, ZIP code=95125, Transaction year=2016, Transaction price=841500),
 Row(_c0=4, Parcel ID=u'429-01-011', Lat=37.3095539, Lng=-121.896306, Property class=0, Year built=1926, Square feet=960, Bedrooms=2

In [82]:
# type(data_set)
data_set = reduce(lambda x,y: x.drop(y),[headers[1]],data_set)
# del headers[4]
del headers[1]
print headers
print data_set.dtypes

['_c0', 'Lat', 'Lng', 'Property class', 'Year built', 'Square feet', 'Bedrooms', 'Bathrooms', 'ZIP code', 'Transaction year', 'Transaction price']
[('_c0', 'int'), ('Lat', 'double'), ('Lng', 'double'), ('Property class', 'int'), ('Year built', 'int'), ('Square feet', 'int'), ('Bedrooms', 'int'), ('Bathrooms', 'double'), ('ZIP code', 'int'), ('Transaction year', 'int'), ('Transaction price', 'int')]


In [83]:
# used_header = headers[1:-3] + headers[-2:-1]
# print used_header
# print len(used_header)

In [84]:
transformed_df = data_set.rdd.map(lambda row: LabeledPoint(row[-1], Vectors.dense(row[0:-1])))
# trained_data, tested_data = transformed_df.randomSplit([0.7, 0.3], seed = 100)
trained_data, tested_data = transformed_df.randomSplit([0.7, 0.3], seed = 100)
print transformed_df.count()

12626


In [85]:
trained_data.take(10)

[LabeledPoint(917184.0, [0.0,37.3084232,-121.8951796,0.0,1900.0,1327.0,4.0,2.0,95125.0,2015.0]),
 LabeledPoint(841500.0, [3.0,37.3089809,-121.8956939,0.0,1905.0,1044.0,3.0,1.0,95125.0,2016.0]),
 LabeledPoint(375370.0, [4.0,37.3095539,-121.896306,0.0,1926.0,960.0,2.0,1.0,95125.0,2010.0]),
 LabeledPoint(840727.0, [6.0,37.3083563,-121.8948274,0.0,1988.0,2371.0,3.0,2.5,95125.0,2005.0]),
 LabeledPoint(129342.0, [7.0,37.30863,-121.8936671,0.0,2006.0,2122.0,3.0,3.0,95125.0,2011.0]),
 LabeledPoint(115436.0, [8.0,37.3084411,-121.8934057,0.0,1939.0,1093.0,2.0,1.0,95125.0,2013.0]),
 LabeledPoint(405846.0, [9.0,37.3083322,-121.8933028,0.0,1939.0,1120.0,2.0,1.0,95125.0,2012.0]),
 LabeledPoint(388350.0, [10.0,37.308102,-121.8930932,0.0,1939.0,1153.0,2.0,1.0,95125.0,2009.0]),
 LabeledPoint(641784.0, [12.0,37.3071787,-121.8919125,0.0,1928.0,1320.0,3.0,1.0,95125.0,2016.0]),
 LabeledPoint(363534.0, [14.0,37.3069323,-121.8922732,0.0,1934.0,989.0,2.0,1.0,95125.0,2010.0])]

In [129]:
labeled_price = headers[-1]
rf = RandomForest.trainRegressor(trained_data, categoricalFeaturesInfo={},
                                    numTrees=8, featureSubsetStrategy="auto",
                                    impurity='variance', maxDepth=8, maxBins=32)
predictions = rf.predict(tested_data.map(lambda x: x.features))
labelsAndPredictions = tested_data.map(lambda lp: lp.label).zip(predictions)
testMSE = labelsAndPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /\
    float(tested_data.count())
print('Test Mean Squared Error = ' + str(math.sqrt(testMSE)))

predictions = rf.predict(trained_data.map(lambda x: x.features))
trainPredictions = trained_data.map(lambda lp: lp.label).zip(predictions)
trainMSE = trainPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /\
    float(tested_data.count())
print('Train Mean Squared Error = ' + str(math.sqrt(trainMSE)))
print('Learned regression forest model:')
print(rf.toDebugString())

Test Mean Squared Error = 171966.148485
Train Mean Squared Error = 245494.836651
Learned regression forest model:
TreeEnsembleModel regressor with 8 trees

  Tree 0:
    If (feature 0 <= 4374.0)
     If (feature 5 <= 2091.0)
      If (feature 8 <= 95112.0)
       If (feature 4 <= 1954.0)
        If (feature 6 <= 2.0)
         If (feature 2 <= -121.878993)
          If (feature 9 <= 2002.0)
           If (feature 2 <= -121.8849346)
            Predict: 139557.0
           Else (feature 2 > -121.8849346)
            Predict: 220358.25
          Else (feature 9 > 2002.0)
           If (feature 0 <= 804.0)
            Predict: 380476.0
           Else (feature 0 > 804.0)
            Predict: 298062.0967741936
         Else (feature 2 > -121.878993)
          If (feature 9 <= 2016.0)
           If (feature 9 <= 2013.0)
            Predict: 251906.72727272726
           Else (feature 9 > 2013.0)
            Predict: 164744.42857142858
          Else (feature 9 > 2016.0)
           Predict: 3

In [124]:
# Save and load model
rf.save(sc, "myrfModel_v3")
print datetime.datetime.now()
sameModel = RandomForestModel.load(sc, "myrfModel_v3")
print datetime.datetime.now()

2017-12-02 21:36:58.046995
2017-12-02 21:36:58.529694


In [128]:
print '****'
print datetime.datetime.now()
#prediction = rf.predict([60.0,37.307741,-121.89593,17108.0,0.0,8400.0,0.0,1938.0,1320.0,6.0,2.0,1.0,95125.0,713.0,753543.2])
# [LabeledPoint(917184.0, [0.0,37.3084232,-121.8951796,0.0,1900.0,1327.0,4.0,2.0,95125.0,2015.0]),
#  LabeledPoint(841500.0, [3.0,37.3089809,-121.8956939,0.0,1905.0,1044.0,3.0,1.0,95125.0,2016.0]),
#  LabeledPoint(375370.0, [4.0,37.3095539,-121.896306,0.0,1926.0,960.0,2.0,1.0,95125.0,2010.0]),
#  LabeledPoint(840727.0, [6.0,37.3083563,-121.8948274,0.0,1988.0,2371.0,3.0,2.5,95125.0,2005.0]),
#  LabeledPoint(129342.0, [7.0,37.30863,-121.8936671,0.0,2006.0,2122.0,3.0,3.0,95125.0,2011.0]),
#  LabeledPoint(115436.0, [8.0,37.3084411,-121.8934057,0.0,1939.0,1093.0,2.0,1.0,95125.0,2013.0]),
#  LabeledPoint(405846.0, [9.0,37.3083322,-121.8933028,0.0,1939.0,1120.0,2.0,1.0,95125.0,2012.0]),
#  LabeledPoint(388350.0, [10.0,37.308102,-121.8930932,0.0,1939.0,1153.0,2.0,1.0,95125.0,2009.0]),
#  LabeledPoint(641784.0, [12.0,37.3071787,-121.8919125,0.0,1928.0,1320.0,3.0,1.0,95125.0,2016.0]),
#  LabeledPoint(363534.0, [14.0,37.3069323,-121.8922732,0.0,1934.0,989.0,2.0,1.0,95125.0,2010.0])]
prediction = sameModel.predict([4.0,37.3095539,-121.896306,0.0,1926.0,960.0,2.0,1.0,95125.0,2010.0])
print datetime.datetime.now()
print prediction

****
2017-12-02 21:38:41.273849
2017-12-02 21:38:41.283298
446790.17253
